<a href="https://colab.research.google.com/github/akankshakusf/Project-DeepLearning-Human-Emotions-Detection-Model/blob/master/DP_Human_Emotion_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# **Import Packages and Libraries**

In [ ]:
#Import packages
import numpy as np
from datetime import datetime
import pathlib
import io
import os
import time
import random
from google.colab import files
import sklearn
import cv2 #computer vision
from sklearn.metrics import confusion_matrix, roc_curve
import seaborn as sns
import matplotlib.pyplot as plt

#tensorflow packages
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import (GlobalAveragePooling2D, Activation, MaxPooling2D, Add, Conv2D, MaxPool2D, Dense,
                                     Flatten, InputLayer, BatchNormalization, Input, Embedding, Permute,
                                     Dropout, RandomFlip, RandomRotation, LayerNormalization, MultiHeadAttention,
                                     RandomContrast, Rescaling, Resizing, Reshape)
from tensorflow.keras.losses import BinaryCrossentropy,CategoricalCrossentropy, SparseCategoricalCrossentropy
from tensorflow.keras.metrics import Accuracy,TopKCategoricalAccuracy, CategoricalAccuracy, SparseCategoricalAccuracy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import (Callback, CSVLogger, EarlyStopping, LearningRateScheduler,
                                        ModelCheckpoint, ReduceLROnPlateau)
from tensorflow.keras.regularizers  import L2, L1

# Define main_directory Structure

In [ ]:
train_directory="/content/dataset/Emotions Dataset/Emotions Dataset/train"
val_directory = "/content/dataset/Emotions Dataset/Emotions Dataset/test"

## Params Dictionary

In [ ]:

CONFIGURATION={
        "BATCH_SIZE":32,
        "IM_SIZE":256,
        "LEARNING_RATE":0.001,
        "N_EPOCHS":10,
        "DROPOUT_RATE":0.0,
        "REGULARIZATION_RATE":0.0,
        "N_FILTERS":6,
        "KERNEL_SIZE":3,
        "N_STRIDES":1,
        "POOL_SIZE":2,
        "N_DENSE_1":100,
        "N_DENSE_2":10,
        "NUM_CLASSES":3,
        "CLASS_NAMES": ["angry", "happy", "sad"],

}


# Data Management

## Data Downloading- Importing Kaggle Dataset

In [ ]:
!pip install -q kaggle

In [ ]:
import os
import zipfile

# Make a hidden .kaggle folder
os.makedirs("/root/.kaggle", exist_ok=True)

# Move kaggle.json to the folder
!mv kaggle.json /root/.kaggle/

# Set permissions
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d muhammadhananasghar/human-emotions-datasethes

In [ ]:
!unzip "/content/human-emotions-datasethes.zip" -d "/content/dataset/"

## Dataset Loading

In [ ]:
train_dataset= tf.keras.preprocessing.image_dataset_from_directory(
    train_directory,
    labels='inferred',
    label_mode='categorical', #one hot endoded
    class_names=CONFIGURATION["CLASS_NAMES"],
    color_mode='rgb',
    batch_size=32,
    image_size=(CONFIGURATION["IM_SIZE"], CONFIGURATION["IM_SIZE"]),
    shuffle=True,
    seed=99,
)

In [ ]:
val_dataset= tf.keras.preprocessing.image_dataset_from_directory(
    val_directory,
    labels='inferred',
    label_mode='categorical', #one hot endoded
    class_names=CONFIGURATION["CLASS_NAMES"],
    color_mode='rgb',
    batch_size=CONFIGURATION["BATCH_SIZE"],
    image_size=(CONFIGURATION["IM_SIZE"], CONFIGURATION["IM_SIZE"]),
    shuffle=True,
    seed=99,
)

In [ ]:
for i in val_dataset.take(1):
  print(i)

## Dataset Visualization

In [ ]:
# Get one batch
images, labels = next(iter(train_dataset))
plt.figure(figsize=(12, 12))

for i in range(16):
    plt.subplot(4, 4, i+1)
    plt.imshow(images[i].numpy() / 255.0)  # normalize and convert tensor to numpy
    plt.title(CONFIGURATION["CLASS_NAMES"][tf.argmax(labels[i], axis=0).numpy()])  # convert one-hot to class index
    plt.axis('off')

plt.tight_layout()
plt.show()

## Data Augmentation

- reference: https://www.tensorflow.org/api_docs/python/tf/keras/layers

In [ ]:
# 1.Augmented layer with "layers method" rotation/flip/contrast
augment_layers = tf.keras.Sequential([
       RandomRotation(factor = (0.25, 0.2501),),
       RandomFlip(mode='horizontal',),
       RandomContrast(factor=0.1),
])

In [ ]:
def augment_layer(image,label):
  return augment_layers(image, training=True), label

In [ ]:
## 2.Augmented layer with "layers method" for resizing and rescaling
resize_rescale_layers = tf.keras.Sequential([
    Resizing(CONFIGURATION["IM_SIZE"], CONFIGURATION["IM_SIZE"]),
    Rescaling(1.0 / 255),
])

## Dataset Preparation

- let tf pre fetch batches in the background while the model is training on the current batch
- let tf automatically choose the optimal number of batches to prefetch based on system performance.

In [ ]:
#prefetch and autotune
train_dataset=(train_dataset
               .map(augment_layer, num_parallel_calls=tf.data.AUTOTUNE)
               .prefetch(tf.data.AUTOTUNE))

In [ ]:
val_dataset=(val_dataset
             .prefetch(tf.data.AUTOTUNE))

In [ ]:
train_dataset

In [ ]:
val_dataset

# Modelling

## Simple Sequential LeNet Model

In [ ]:
# clear up session cache
from tensorflow.keras import backend as K
# Clear the previous session to reset layer count
K.clear_session()


In [ ]:
'''
1. Instantiate the CNN model "Simple Sequential Model"
'''

lenet_model = tf.keras.Sequential([

    #InputLayer(shape = (None, None, 3), ),
    InputLayer(input_shape = (CONFIGURATION["IM_SIZE"], CONFIGURATION["IM_SIZE"],3)),

    resize_rescale_layers, # embedding resize and rescale into SequentialAPI
    augment_layers,

    # 1st conv layer (extracts basic patterns)
    Conv2D(filters=CONFIGURATION["N_FILTERS"], kernel_size=CONFIGURATION["KERNEL_SIZE"],
           strides=CONFIGURATION["N_STRIDES"],
           padding="valid", activation="relu",
           kernel_regularizer=L2(CONFIGURATION["REGULARIZATION_RATE"])),

    BatchNormalization(),
    MaxPool2D(pool_size=CONFIGURATION["POOL_SIZE"],
              strides=CONFIGURATION["N_STRIDES"]*2),  # Downsamples feature maps

    Dropout(rate=CONFIGURATION["DROPOUT_RATE"]),   #add a dropout layer

    # 2nd conv layer (extracts deeper features)
    Conv2D(filters=CONFIGURATION["N_FILTERS"]*2+4, kernel_size=CONFIGURATION["KERNEL_SIZE"],
           strides=CONFIGURATION["N_STRIDES"],
           padding="valid", activation="relu",
           kernel_regularizer=L2(CONFIGURATION["REGULARIZATION_RATE"])),

    BatchNormalization(),
    MaxPool2D(pool_size=CONFIGURATION["POOL_SIZE"], strides=CONFIGURATION["N_STRIDES"]*2),  # Downsampling again


    Flatten(),  # Converts 2D feature maps into 1D array

    Dense(CONFIGURATION["N_DENSE_1"], activation="relu",kernel_regularizer=L2(CONFIGURATION["REGULARIZATION_RATE"])),  # Fully connected layer
    BatchNormalization(),
    Dropout(rate=CONFIGURATION["DROPOUT_RATE"]),   #add a dropout layer

    Dense(CONFIGURATION["N_DENSE_2"], activation="relu",
          kernel_regularizer=L2(CONFIGURATION["REGULARIZATION_RATE"])),   # Further processing
    BatchNormalization(),

    Dense(CONFIGURATION["NUM_CLASSES"], activation="softmax"),     # Output layer (multi-class classification)

])

# Print model summary
lenet_model.summary()



# Model Training


In [ ]:
loss_function = CategoricalCrossentropy()
#loss_function = SparseCategoricalCrossentropy()

In [ ]:
metrics = [CategoricalAccuracy(name = "accuracy"), TopKCategoricalAccuracy(k=2, name = "top_k_accuracy")]

In [ ]:
#compile the model
lenet_model.compile(
    optimizer=Adam(learning_rate=CONFIGURATION["LEARNING_RATE"]),
    loss=loss_function,
    metrics=metrics)

In [ ]:
#fit the model
history = lenet_model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=CONFIGURATION["N_EPOCHS"],
    verbose=True)

# Visualization

In [ ]:
# Model Accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['accuracy', 'val_accuracy'])
plt.show()

In [ ]:
# Model Loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Losses')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['train_loss', 'val_loss'])
plt.show()

# Model Evaluation

In [ ]:
#evaluate model on val data
lenet_model.evaluate(val_dataset)

# Model Testing

- 1. testing for happy image

In [ ]:
#test an image
test_image = cv2.imread("/content/dataset/Emotions Dataset/Emotions Dataset/test/happy/111073.jpg")

# Convert the image to tensors
im = tf.constant(test_image, dtype=tf.float32)
print(im.shape) # but this needs to be 4 dimensional


#adding batch dimension to shape [1, height, width, channels])
im = tf.expand_dims(im, axis=0)
print(im.shape)

#get predictions (argmax: get max probabilities)
predictions= lenet_model(im)
print(CONFIGURATION["CLASS_NAMES"][tf.argmax(predictions, axis=-1).numpy()[0]])

- 2. testing for sad image

In [ ]:
#test an image
test_image = cv2.imread("/content/dataset/Emotions Dataset/Emotions Dataset/test/sad/105565.jpg_brightness_1.jpg")

# Convert jpg to tensors
im = tf.constant(test_image, dtype=tf.float32)
print(im.shape) # but this needs to be 4 dimensional

#adding batch dimension to shape [1, height, width, channels])
im = tf.expand_dims(im, axis=0)
print(im.shape)

#get predictions (argmax: get max probabilities)
predictions= lenet_model(im)
print(CONFIGURATION["CLASS_NAMES"][tf.argmax(predictions, axis=-1).numpy()[0]])

In [ ]:
# Get one batch
images, labels = next(iter(train_dataset))
plt.figure(figsize=(9, 9))

for i in range(16):
    plt.subplot(4, 4, i+1)
    plt.imshow(images[i].numpy()/ 255.0)  # normalize and convert tensor to numpy

    #titles on images
    plt.title("True Labels: " + CONFIGURATION["CLASS_NAMES"][tf.argmax(labels[i], axis=0).numpy()]+ "\n" +
              "Predicted Labels: " + CONFIGURATION["CLASS_NAMES"][tf.argmax(lenet_model(tf.expand_dims(images[i], axis=0)), axis=-1).numpy()[0]])

plt.tight_layout()
plt.show()

# Confusion Matrix

In [ ]:
predicted= []
labels= []

#iterate through the validation ds
for im, label in val_dataset:
  predicted.append(lenet_model(im))
  labels.append(label.numpy())

In [ ]:
#leave the last batch of 32 and take rest & later flatten the batches

#leaving last batch for now [:-1]
print(np.argmax(labels[:-1], axis= -1).flatten())

#taking just last batch [-1]
print(np.argmax(labels[-1], axis= -1).flatten())

#for labels combining last batch and all batch through concatenation
print(np.concatenate([np.argmax(labels[:-1], axis= -1).flatten(),np.argmax(labels[-1], axis= -1).flatten()]))

#for predicted values combining last batch and all batch through concatenation
print(np.concatenate([np.argmax(predicted[:-1], axis= -1).flatten(),np.argmax(predicted[-1], axis= -1).flatten()]))

In [ ]:
# predicted and true values
predicted = np.concatenate([np.argmax(predicted[:-1], axis= -1).flatten(),np.argmax(predicted[-1], axis= -1).flatten()]) #what model predicted
labels = np.concatenate([np.argmax(labels[:-1], axis= -1).flatten(),np.argmax(labels[-1], axis= -1).flatten()]) #true values


In [ ]:
plt.figure(figsize=(4,4))
cm=confusion_matrix(predicted,labels)
print(cm)

sns.heatmap(cm,annot=True, fmt='g',cmap="crest")
plt.title("Confusion Matrix")
plt.ylabel("Actual")
plt.xlabel("Predicted")

-The model performed best in recognizing happy, correctly classifying 879 samples. However, it misclassified 132 as angry and 214 as sad.

-For the sad class, the model correctly identified 475 cases, while incorrectly classifying 87 as angry and 93 as happy.

-Recognition of the angry class was weakest, correctly classifying only 294 cases, with 31 misclassified as happy and 67 as sad.

-The large off-diagonal values in the "happy" column indicate a bias towards predicting "happy" more frequently, even when the actual emotion is "angry" or "sad".